In [1]:
# ソースからのHuggingface Transformersのインストール
!git clone https://github.com/huggingface/transformers -b v4.4.2
!pip install transformers
# Huggingface Datasetsのインストール
!pip install datasets==1.2.1
!pip install rouge_score
!pip install sentencepiece

Cloning into 'transformers'...
remote: Enumerating objects: 84608, done.
remote: Counting objects: 100% (117/117), done.
remote: Compressing objects: 100% (68/68), done.
remote: Total 84608 (delta 53), reused 78 (delta 33), pack-reused 84491
Receiving objects: 100% (84608/84608), 67.51 MiB | 4.87 MiB/s, done.
Resolving deltas: 100% (60778/60778), done.
Note: switching to '9f43a425fe89cfc0e9b9aa7abd7dd44bcaccd79a'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by switching back to a branch.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -c with the switch command. Example:

  git switch -c <new-branch-name>

Or undo this operation with:

  git switch -

Turn off this advice by setting config variable advice.detachedHead to false

     |████████████████████████████████| 159 kB 1.9 MB/s eta 0:00:

     |████████████████████████████████| 508 kB 2.2 MB/s eta 0:00:01
  ERROR: Command errored out with exit status 1:
   command: /Users/yuto/miniforge3/envs/py38/bin/python3.8 -u -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'/private/var/folders/n0/sq44y54n4rq_98y9bm_q__j80000gn/T/pip-install-lrg_aykm/sentencepiece_785bae5942044f09ba1270d034499bc4/setup.py'"'"'; __file__='"'"'/private/var/folders/n0/sq44y54n4rq_98y9bm_q__j80000gn/T/pip-install-lrg_aykm/sentencepiece_785bae5942044f09ba1270d034499bc4/setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d /private/var/folders/n0/sq44y54n4rq_98y9bm_q__j80000gn/T/pip-wheel-7ybse70o
       cwd: /private/var/folders/n0/sq44y54n4rq_98y9bm_q__j80000gn/T/pip-install-lrg_aykm/sentencepiece_785bae594

In [7]:
%%time

# ファインチューニングの実行
!python ./transformers/examples/seq2seq/run_summarization.py \
    --model_name_or_path=sonoisa/t5-base-japanese \
    --do_train \
    --do_eval \
    --train_file=train_data.csv \
    --validation_file=test_data.csv \
    --per_device_train_batch_size=4 \
    --per_device_eval_batch_size=4 \
    --save_steps=5000 \
    --save_total_limit=3 \
    --output_dir=output/ \
    --predict_with_generate \
    --use_fast_tokenizer=False

09/21/2021 03:16:42 - WARNING - __main__ -   Process rank: -1, device: cpu, n_gpu: 0distributed training: False, 16-bits training: False
09/21/2021 03:16:42 - INFO - __main__ -   Training/evaluation parameters Seq2SeqTrainingArguments(
_n_gpu=0,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_find_unused_parameters=None,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=False,
do_train=True,
eval_accumulation_steps=None,
eval_steps=None,
evaluation_strategy=IntervalStrategy.NO,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
generation_max_length=None,
generation_num_beams=None,
gradient_accumulation_steps=1,
greater_is_better=None,
group_by_length=False,
ignore_data_skip=False,
label_names=None,
label_smoothing_factor=0.0,
learning_rate=5e-05,
length_column_name=length,
load_best_model_at_end=False,
local_rank=-1,
log_level=-1,
log_l

 44%|████████████████▎                    | 305/690 [2:16:04<3:20:56, 31.32s/it]^C
CPU times: user 1min 42s, sys: 35.8 s, total: 2min 18s
Wall time: 2h 19min 22s


In [ ]:
import torch
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM

# モデルとトークナイザーの準備
model = AutoModelForSeq2SeqLM.from_pretrained('output/')    
tokenizer = AutoTokenizer.from_pretrained('sonoisa/t5-base-japanese') 

# テキスト
text = r"""
地球の人里離れた山奥に住む尻尾の生えた少年・孫悟空はある日、西の都からやって来た少女ブルマと出会う。そこで、7つ集めると神龍が現れ、どんな願いでも一つだけ叶えてくれるというドラゴンボールの存在を、さらに育ての親である孫悟飯の形見として大切に持っていた球がその1つ「四星球」であることを知り、ブルマと共に残りのドラゴンボールを探す旅に出る。人さらいのウーロンや盗賊のヤムチャなどを巻き込んだボール探しの末、世界征服を企むピラフ一味にボールを奪われ神龍を呼び出されるが、ウーロンがとっさに言い放った下らない願いを叶えてもらうことで一味の野望を阻止する。
"""

# テキストをテンソルに変換
inputs = tokenizer.encode(text, return_tensors="pt", max_length=512, truncation=True)

# 推論
model.eval()
with torch.no_grad():
    summary_ids = model.generate(inputs) #, max_length=512, min_length=5, length_penalty=5., num_beams=2)
    summary = tokenizer.decode(summary_ids[0])
    print(summary)